In [1]:
import theano as th
import theano.tensor as T

In [2]:
a = T.scalar()
b = T.scalar()
c = a*b

f = th.function([a,b],c)

f(2,3)

array(6.0)

In [28]:
state = th.shared(1)
inc = T.iscalar()
dec = T.iscalar()

accumulate = th.function([inc],state,updates=[(state,state+inc)],on_unused_input='ignore')
decrement = th.function([dec],state,updates=[(state,state-dec)])

In [29]:
print(accumulate(10))
print(accumulate(5))
print(decrement(3))
print(state.get_value())

1
11
16
13


In [30]:
newstate = th.shared(2)
new_acc = accumulate.copy(swap={state:newstate})
print(new_acc(10))
print(new_acc(5))
print(newstate.get_value())

[array(2)]
[array(12)]
17


In [31]:
null_acc = accumulate.copy(delete_updates=True)
null_acc(9000)
print(state.get_value())

13


In [34]:
from theano.tensor.shared_randomstreams import RandomStreams

srng = RandomStreams(seed=123)
uniform = srng.uniform((2,3))

rngf = th.function([],uniform)
rngf_static = th.function([],uniform,no_default_updates=True)

print(rngf())
print(rngf_static())
print(rngf_static())

[[ 0.72803009  0.59044123  0.23715077]
 [ 0.69958932  0.86608782  0.35008632]]
[[ 0.37173976  0.5594106   0.33626034]
 [ 0.62003789  0.81396029  0.22291474]]
[[ 0.37173976  0.5594106   0.33626034]
 [ 0.62003789  0.81396029  0.22291474]]


Note how static version won't update rng's state.

Oh and, all references to one random variable produce same value during one function evaluation.

In [37]:
nearly_zeros = th.function([], uniform + uniform - 2 * uniform)
nearly_zeros()

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.]])